### DESCRIPTION
Script to compute demographic and morphological traits such as biovolume (µm3), surface area (µm2), surface/volume ratio, density (cells/L), cell carbon content (pg), total biovolume (µm³/L), total carbon content (pg/L). 
### INPUT VARIABLES
- datain: the input file
- CalcType: the computation type simplified or advanced
- CompTraits: one or more among biovolume,totalbiovolume,density,surfacearea,surfacevolumeratio,cellcarboncontent,totalcarboncontent
- CountingStrategy: one among density0,density1,density2,density3 

### OUTPUT 
file in .csv format, containing the original input data and the new calculated traits

In [14]:
# input parameters
param_datain <- '~/Unisalento/Lifewatch/Phyto_VRE/Input/Phytoplankton__Progetto_Strategico_2009_2012_Australia.csv'

param_CalcType <- 'advanced'

param_biovolume <- 1 # if 1 it is calculated, if 0 it is not calculated
param_totalbiovolume <- 1
param_density <- 1
param_surfacearea <- 1
param_surfacevolumeratio <- 1
param_cellcarboncontent <- 1
param_totalcarboncontent <- 1

if (param_density==1) {param_CountingStrategy <- 'density0'}

# read input cvs
df.datain=read.csv(param_datain,stringsAsFactors=F,sep = ";", dec = ".")
df.datain$measurementremarks <- tolower(df.datain$measurementremarks) # eliminate capital letters
df.datain$index <- c(1:nrow(df.datain)) # needed to restore rows order later

# read support cvs file
df.operator<-read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/2_FILEinformativo_OPERATORE.csv',stringsAsFactors=F,sep = ";", dec = ".") ## load internal database 
df.operator[df.operator==('no')]<-NA
df.operator[df.operator==('see note')]<-NA

# merge dataframes
df.merged <- merge(x = df.datain, y = df.operator, by = c("scientificname","measurementremarks"), all.x = TRUE)

# check if mandatory fields are present
if(!'diameterofsedimentationchamber'%in%names(df.merged))df.merged$diameterofsedimentationchamber=NA
if(!'diameteroffieldofview'%in%names(df.merged))df.merged$diameteroffieldofview=NA
if(!'transectcounting'%in%names(df.merged))df.merged$transectcounting=NA
if(!'numberofcountedfields'%in%names(df.merged))df.merged$numberofcountedfields=df.merged$transectcounting
if(!'numberoftransects'%in%names(df.merged))df.merged$numberoftransects==df.merged$transectcounting
if(!'settlingvolume'%in%names(df.merged))df.merged$settlingvolume=NA
if(!'dilutionfactor'%in%names(df.merged))df.merged$dilutionfactor=1

# save merged dataframe as csv
write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F)
write.table(df.datain,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfdatain.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F) 

logical(0)

In [15]:
df.merged=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',stringsAsFactors=F,sep = ";", dec = ".")
CalcType=param_CalcType

if(CalcType=='advanced'){
  df.merged.concat <- df.merged[is.na(df.merged$formulaformissingdimension),]
  md.formulas <- unique(df.merged[!is.na(df.merged$formulaformissingdimension),]$formulaformissingdimension)
  for(md.form in md.formulas){
    df.temp <- subset(df.merged,formulaformissingdimension==md.form)
    for(md in unique(df.temp$missingdimension)){
      df.temp2 <- df.temp[df.temp$missingdimension==md,]
      df.temp2[[md]] <- round(with(df.temp2,eval(parse(text=md.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp2)
    }
  }
  df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
  df.merged <- df.merged.concat
} else if(CalcType=='simplified'){
  df.merged.concat <- df.merged[is.na(df.merged$formulaformissingdimensionsimplified),]
  md.formulas <- unique(df.merged[!is.na(df.merged$formulaformissingdimensionsimplified),]$formulaformissingdimensionsimplified)
  for(md.form in md.formulas){
    df.temp <- subset(df.merged,formulaformissingdimensionsimplified==md.form)
    for(md in unique(df.temp$missingdimensionsimplified)){
      df.temp2 <- df.temp[df.temp$missingdimensionsimplified==md,]
      df.temp2[[md]] <- round(with(df.temp2,eval(parse(text=md.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp2)
    }
  }
  df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
  df.merged <- df.merged.concat
}
    
write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F)    


In [16]:
df.merged=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',stringsAsFactors=F,sep = ";", dec = ".")
TraitBiovolume=param_biovolume
CalcType=param_CalcType

if(TraitBiovolume==1){
  
  if(CalcType=='advanced'){
    df.merged$biovolume <- rep(NA,length=nrow(df.merged))
    df.merged.concat <- df.merged[is.na(df.merged$formulaforbiovolume),]
    bv.formulas <- unique(df.merged[!is.na(df.merged$formulaforbiovolume),]$formulaforbiovolume)
    for(bv.form in bv.formulas){
      df.temp <- subset(df.merged,formulaforbiovolume==bv.form)
      df.temp$biovolume <- round(with(df.temp,eval(parse(text=bv.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
    df.merged <- df.merged.concat
    BV_calc = df.merged$biovolume
  }
  else if(CalcType=='simplified'){
    df.merged$biovolume <- rep(NA,length=nrow(df.merged))
    df.merged.concat <- df.merged[is.na(df.merged$formulaforbiovolumesimplified),]
    bv.formulas <- unique(df.merged[!is.na(df.merged$formulaforbiovolumesimplified),]$formulaforbiovolumesimplified)
    for(bv.form in bv.formulas){
      df.temp <- subset(df.merged,formulaforbiovolumesimplified==bv.form)
      df.temp$biovolume <- round(with(df.temp,eval(parse(text=bv.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
    df.merged <- df.merged.concat
    BV_calc = df.merged$biovolume
  }
} 

write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F)

In [17]:
df.merged=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',stringsAsFactors=F,sep = ";", dec = ".")

TraitCellcarboncontent=param_cellcarboncontent
TraitBiovolume=param_biovolume

if(TraitCellcarboncontent==1){
  
  df.merged$cellcarboncontent <- rep(NA,length=nrow(df.merged))
  if(TraitBiovolume==1){
    df.merged.concat <- df.merged[is.na(df.merged$biovolume),]
    df.cc <- df.merged[!is.na(df.merged$biovolume),]
    df.cc1 <- subset(df.cc,biovolume <= 3000)
    df.cc2 <- subset(df.cc,biovolume > 3000)
    cc.formulas1 <- unique(df.merged[!is.na(df.merged$formulaforweight1),]$formulaforweight1)
    for(cc.form in cc.formulas1){
      df.temp <- subset(df.cc1,formulaforweight1==cc.form)
      df.temp$cellcarboncontent <- round(with(df.temp,eval(parse(text=tolower(cc.form)))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    cc.formulas2 <- unique(df.merged[!is.na(df.merged$formulaforweight2),]$formulaforweight2)
    for(cc.form in cc.formulas2){
      df.temp <- subset(df.cc2,formulaforweight2==cc.form)
      df.temp$cellcarboncontent <- round(with(df.temp,eval(parse(text=tolower(cc.form)))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
    df.merged <- df.merged.concat
    CC_calc = df.merged$cellcarboncontent
  }
}
    
write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F) 

In [18]:
df.merged=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',stringsAsFactors=F,sep = ";", dec = ".")

TraitDensity=param_density
CountingStrategy=param_CountingStrategy

if(TraitDensity==1){
  df.merged$density <- rep(NA,length=nrow(df.merged))
  # default method to calculate the density
  if(CountingStrategy=='density0'){  
    df.merged.concat <- df.merged[(is.na(df.merged$volumeofsedimentationchamber)) & (is.na(df.merged$transectcounting)),]
    df.temp <- df.merged[!is.na(df.merged$volumeofsedimentationchamber) & !is.na(df.merged$transectcounting),]
    df.temp1 <- subset(df.temp,volumeofsedimentationchamber <= 5)
    df.temp1$density <- df.temp1$organismquantity/df.temp1$transectcounting*1000/0.001979
    df.merged.concat <- rbind(df.merged.concat,df.temp1)
    df.temp2 <- subset(df.temp,(volumeofsedimentationchamber > 5) & (volumeofsedimentationchamber <= 10))
    df.temp2$density <- df.temp2$organismquantity/df.temp2$transectcounting*1000/0.00365
    df.merged.concat <- rbind(df.merged.concat,df.temp2)
    df.temp3 <- subset(df.temp,(volumeofsedimentationchamber > 10) & (volumeofsedimentationchamber <= 25))
    df.temp3$density <- df.temp3$organismquantity/df.temp3$transectcounting*1000/0.010555
    df.merged.concat <- rbind(df.merged.concat,df.temp3)
    df.temp4 <- subset(df.temp,(volumeofsedimentationchamber > 25) & (volumeofsedimentationchamber <= 50))
    df.temp4$density <- df.temp4$organismquantity/df.temp4$transectcounting*1000/0.021703
    df.merged.concat <- rbind(df.merged.concat,df.temp4)
    df.temp5 <- subset(df.temp,volumeofsedimentationchamber > 50)
    df.temp5$density <- df.temp5$organismquantity/df.temp5$transectcounting*1000/0.041598
    df.merged.concat <- rbind(df.merged.concat,df.temp5)
    df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
    df.merged <- df.merged.concat
    D_calc <- round(df.merged$density,2)
  }
  # counts per random field
  else if(CountingStrategy=='density1'){
    df.merged$areaofsedimentationchamber <- ((df.merged$diameterofsedimentationchamber/2)^2)*pi
    df.merged$areaofcountingfield <- ((df.merged$diameteroffieldofview/2)^2)*pi
    df.merged$density <- round(df.merged$organismquantity*1000*df.merged$areaofsedimentationchamber/df.merged$numberofcountedfields*df.merged$areaofcountingfield*df.merged$settlingvolume,2)
  }
  # counts per diameter transects
  else if(CountingStrategy=='density2'){
    df.merged$density <- round(((df.merged$organismquantity/df.merged$numberoftransects)*(pi/4)*(df.merged$diameterofsedimentationchamber/df.merged$diameteroffieldofview))*1000/df.merged$settlingvolume,2)
  }
  # counting method for whole chamber
  else if(CountingStrategy=='density3'){
    df.merged$density <- round((df.merged$organismquantity*1000)/df.merged$settlingvolume,2)
  }
  D_calc = df.merged$density/df.merged$dilutionfactor
}
      
    
write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F)


In [19]:
df.merged=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',stringsAsFactors=F,sep = ";", dec = ".")

TraitTotalbiovolume=param_totalbiovolume
TraitDensity=param_density
TraitBiovolume=param_biovolume

if(TraitTotalbiovolume==1){
  if((TraitDensity==0) & (!'density'%in%names(df.merged))) df.merged$density<-NA
  if((TraitBiovolume==0) & (!'biovolume'%in%names(df.merged))) df.merged$biovolume<-NA
  TBV_calc = round(df.merged$density*df.merged$biovolume,2)
}

write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F)


In [20]:
df.merged=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',stringsAsFactors=F,sep = ";", dec = ".")

TraitSurfacearea=param_surfacearea
CalcType=param_CalcType

if(TraitSurfacearea==1){
  if(CalcType=='advanced'){
    df.merged$surfacearea <- rep(NA,length=nrow(df.merged))
    df.merged.concat <- df.merged[is.na(df.merged$formulaforsurface),]
    sa.formulas <- unique(df.merged[!is.na(df.merged$formulaforsurface),]$formulaforsurface)
    for(sa.form in sa.formulas){
      df.temp <- subset(df.merged,formulaforsurface==sa.form)
      df.temp$surfacearea <- round(with(df.temp,eval(parse(text=sa.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
    df.merged <- df.merged.concat
    SA_calc <- df.merged$surfacearea
  }
  else if(CalcType=='simplified'){
    df.merged$surfacearea <- rep(NA,length=nrow(df.merged))
    df.merged.concat <- df.merged[is.na(df.merged$formulaforsurfacesimplified),]
    sa.formulas <- unique(df.merged[!is.na(df.merged$formulaforsurfacesimplified),]$formulaforsurfacesimplified)
    for(sa.form in sa.formulas){
      df.temp <- subset(df.merged,formulaforsurfacesimplified==sa.form)
      df.temp$surfacearea <- round(with(df.temp,eval(parse(text=sa.form))),2)
      df.merged.concat <- rbind(df.merged.concat,df.temp)
    }
    df.merged.concat <- df.merged.concat[order(df.merged.concat$index),]
    df.merged <- df.merged.concat
    SA_calc <- df.merged$surfacearea
  }
}

write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F)


Warning message in sqrt((h^2) - (d^2)):
"Si è prodotto un NaN"
Warning message in sqrt((h^2) - (d^2)):
"Si è prodotto un NaN"
Warning message in sqrt(4 * h^2 - ((b + c)^2)):
"Si è prodotto un NaN"
Warning message in sqrt(4 * h^2 - ((b + c)^2)):
"Si è prodotto un NaN"


In [21]:
df.merged=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',stringsAsFactors=F,sep = ";", dec = ".")

TraitSurfacevolumeratio=param_surfacevolumeratio
TraitSurfacearea=param_surfacearea
TraitBiovolume=param_biovolume

if(TraitSurfacevolumeratio==1){
  if((TraitSurfacearea==0) & (!'surfacearea'%in%names(df.merged))) df.merged$surfacearea<-NA
  if((TraitBiovolume==0) & (!'biovolume'%in%names(df.merged))) df.merged$biovolume<-NA
  SVR_calc<-round(df.merged$surfacearea/df.merged$biovolume,2)
}

write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F)


In [22]:
df.merged=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',stringsAsFactors=F,sep = ";", dec = ".")

TraitTotalcarboncontent=param_totalcarboncontent
TraitDensity=param_density
TraitCellcarboncontent=param_cellcarboncontent

if(TraitTotalcarboncontent==1){
  if((TraitDensity==0) & (!'density'%in%names(df.merged))) df.merged$density<-NA
  if((TraitCellcarboncontent==0) & (!'cellcarboncontent'%in%names(df.merged))) df.merged$cellcarboncontent<-NA
  TCC_calc<-round(df.merged$density*df.merged$cellcarboncontent,2)
}

write.table(df.merged,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfmerged.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F)


In [23]:
df.datain=read.csv('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/dfdatain.csv',stringsAsFactors=F,sep = ";", dec = ".")

BV=TraitBiovolume 
TBV=TraitTotalbiovolume
D=TraitDensity
SA=TraitSurfacearea
SVR=TraitSurfacevolumeratio
CC=TraitCellcarboncontent
TCC=TraitTotalcarboncontent

if(BV==1) {
    BV_column=BV_calc
    if('biovolume'%in%names(df.datain)) df.datain<-subset(df.datain,select=-biovolume) # drop column if already present
    df.datain$biovolume <- BV_column # write column with the results at the end of the dataframe
    }
if(CC==1) {
    CC_column=CC_calc
    if('cellcarboncontent'%in%names(df.datain)) df.datain<-subset(df.datain,select=-cellcarboncontent)
    df.datain$cellcarboncontent <- CC_column
    }
if(D==1) {
    D_column=D_calc
    df.datain$density <- D_column
    }
if(TBV==1) {
    TBV_column=TBV_calc
    df.datain$totalbiovolume <- TBV_column
    }
if(SA==1) {
    SA_column=SA_calc
    if('surfacearea'%in%names(df.datain)) df.datain<-subset(df.datain,select=-surfacearea)
    df.datain$surfacearea <- SA_column
    }
if(SVR==1) {
    SVR_column=SVR_calc
    df.datain$surfacevolumeratio <- SVR_column
    }
if(TCC==1) {
    TCC_column=TCC_calc
    df.datain$totalcarboncontent <- TCC_column
    }

df.datain <- subset(df.datain,select = -index)

write.table(df.datain,paste('~/Unisalento/Lifewatch/Phyto_VRE/Script_R/Traits_Computation/Output/TraitsOutput.csv',sep=''),row.names=F,sep = ";",dec = ".",quote=F) 